In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [18]:
df = pd.read_csv('medical_examination.csv')

In [19]:
df

,id,age,sex,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


Add an overweight column to the data. To determine if a person is overweight, first calculate their BMI by dividing their weight in kilograms by the square of their height in meters. If that value is > 25 then the person is overweight. Use the value 0 for NOT overweight and the value 1 for overweight.

In [20]:
# Adding Column for the Square Root of Height
df['sq_height_m'] = (np.sqrt(df['height']))/100

# Adding Column for Body Mass Index
df['bmi'] = df['weight']/df['sq_height_m']

df
# First get 

,id,age,sex,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,sq_height_m,bmi
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0,0.129615,478.340385
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1,0.124900,680.544654
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1,0.128452,498.239324
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1,0.130000,630.769231
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0,0.124900,448.358831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0,0.129615,586.352730
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1,0.125698,1002.402178
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1,0.135277,776.182335
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1,0.127671,563.947524


In [4]:
# Add 'overweight' column
df['overweight'] = None

Normalize data by making 0 always good and 1 always bad.
If the value of 'cholesterol' or 'gluc' is 1, make the value 0. 
If the value is more than 1, make the value 1.


Convert the data into long format and create a chart that shows the value counts of the categorical features using seaborn's catplot(). The dataset should be split by 'Cardio' so there is one chart for each cardio value. The chart should look like examples/Figure_1.png.

Clean the data. Filter out the following patient segments that represent incorrect data:
diastolic pressure is higher than systolic (Keep the correct data with (df['ap_lo'] <= df['ap_hi']))
height is less than the 2.5th percentile (Keep the correct data with (df['height'] >= df['height'].quantile(0.025)))
height is more than the 97.5th percentile
weight is less than the 2.5th percentile
weight is more than the 97.5th percentile

Create a correlation matrix using the dataset. Plot the correlation matrix using seaborn's heatmap(). Mask the upper triangle. The chart should look like examples/Figure_2.png.

Any time a variable is set to None, make sure to set it to the correct code

In [6]:
# Draw Categorical Plot
def draw_cat_plot():
    # Create DataFrame for cat plot using `pd.melt` using just 
    # the values from 'cholesterol', 'gluc', 'smoke', 'alco', 
    # 'active', and 'overweight'.
    df_cat = None
    
    # Group and reformat the data to split it by 'cardio'. Show the counts 
    # of each feature. You will have to rename one of the columns for the 
    # catplot to work correctly.
    
    df_cat = None
    # Do not modify the next two lines
    
    fig.savefig('catplot.png')
    return fig

In [9]:
# Draw Heat Map
def draw_heat_map():
    # Clean the data
    df_heat = None

    # Calculate the correlation matrix
    corr = None

    # Generate a mask for the upper triangle
    mask = None



    # Set up the matplotlib figure
    fig, ax = None

    # Draw the heatmap with 'sns.heatmap()'



    # Do not modify the next two lines
    fig.savefig('heatmap.png')
    return fig